In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
import asyncio

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2025-04-10 12:19:22,396 | INFO     |: Logger demo_notebook.log initialized.


# Start/Stop the Simulator and Plotter

### OPTION 1: GUI
- You can use a GUI to start, stop and track the running status of the simulator and plotter.
- Run the below cell **only once** to render the GUI. You can then interact with the GUI using your mouse.
- The GUI will not respond when the notebook's kernel is busy, just like any other cell.

In [2]:
gui.show()

TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

### OPTION 2: Programmatically
You can also use functions to do the same.

In [3]:
# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

You can also quit each process by closing their respective windows.

### Make sure there is only one running instance of the simulator and plotter to prevent any confusion.

<hr>

# Interactions

## Using your mouse and keyboard

### Simulator
- Press the up and down arrow keys to increase and decrease the linear velocity of the virtual robot, respectively.
- Press the left and right arrow keys to increase to increase and decrease the angular velocity of the virtual robot, respectively.
- Hit spacebar to stop the robot.
- Press h to display the full keyboard map.

### Plotter
- Use the GUI buttons to show/hide the various plots.
- Use your mouse to zoom in and out of the plot.
- Press the "A" button (bottom-left to the plotting box and above "Plotted Points") to auto-fit the plot to your window.



## Using the Commander class
The **Commander** class (defined in *commander.py*) allows you to interact with the simulator and the plotter programmatically. <br>
__NOTE__: The units of measurement are meters and radians.

<table align="left">
    <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Utility Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">sim_is_running()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the run status of the simulator.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plotter_is_running()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the run status of the plotter.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Plotter Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_odom(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in red. Units are (meters, meters).</span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_gt(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in green. Units are (meters, meters).</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_bel(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in blue. Units are (meters, meters).</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_map()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot the map based on the map lines in <em>world.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reset_plotter()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reset the plots in the plotter.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Simulator Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">set_vel(linear_vel, angular_vel)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Set the linear velocity (m/s) and angular velocity (rad/s) of the virtual robot.</span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">get_pose()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the odometry and ground truth poses of the virtual robot as two numpy arrays. The units of each pose are (meters, meters, radians)</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">get_sensor()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the ToF sensor data (in meters) of the virtual robot as a numpy column array.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reset_sim()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reset the virtual robot to its initial pose.</span></th>
    </tr>    
   
</table>


In [25]:
# Reset Plotter and Simulator
cmdr.reset_plotter()
cmdr.reset_sim()

turn_count = 0
move_count = 0
bot_state = 0
max_time = 200;
time = 0;

# Loop for pose
while cmdr.sim_is_running() and cmdr.plotter_is_running() and time < max_time:
    pose, gt_pose = cmdr.get_pose()
    if (move_count > 60):
        move_count = 0
        bot_state = 0
    elif (turn_count > 51):
        turn_count = 0;
        bot_state = 1

    if (bot_state == 1):
        cmdr.set_vel(0.2, 0)
        move_count = move_count + 1
    elif (bot_state ==0):
        cmdr.set_vel(0, -0.9)
        turn_count = turn_count + 1

    cmdr.plot_gt(gt_pose[0], gt_pose[1])
    time = time + 1;
cmdr.set_vel(0, 0)

In [26]:
# Loop for sensor
while cmdr.sim_is_running() and cmdr.plotter_is_running():
    sensor_values = cmdr.get_sensor()
    print(sensor_values)
    await asyncio.sleep(1)

[1.50631312]
[1.47038682]
[1.42125022]
[1.46639419]
[1.41859397]
[1.39321595]
[1.46812302]
[1.48834298]
[1.46824366]
[1.49178626]
[1.47207448]
[1.41623368]
[1.48050825]
[1.45515432]
[1.4776783]
[1.45590135]
[1.46806716]
[1.42671385]
[1.41045383]
[1.40369403]
[1.48594438]
[1.4126849]
[1.41990949]
[1.47041606]
[1.39404683]
[1.44020306]
[1.48217594]
[1.41205966]
[1.44392807]
[1.42820005]
[1.4378459]


CancelledError: 